# 对比

In [44]:
import time
import sys
import os
import numpy as np
import faiss

In [45]:
d = 64                            
nb = 100000                         
nq = 10000                          
np.random.seed(1234)                
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.   
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

## IndexFlatL2

In [46]:
index = faiss.IndexFlatL2(d)        

print('is_trained',index.is_trained)
index.add(xb)                       
print('is_trained',index.is_trained) 

k = 4                               # k=4 k邻近搜索
D, I = index.search(xq, k)          # 执行搜索
print(I[:5])                        # 最初五次查询的结果
print(I[-5:])                       # 最后五次查询的结果

is_trained True
is_trained True
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


# IndexFlatL2 是精确暴力搜索，后面的索引跟它相比即可。

## IndexIVFFlat-速度更快

将数据集分割成多个，我们在d维空间中定义Voronoi单元，每个数据库向量落在其中一个单元格中。

在搜索时，只有查询x所在的单元格中包含的数据库向量y和几个相邻的数据库向量y与查询向量进行比较。

In [47]:
nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
# here we specify METRIC_L2, by default it performs inner-product search

print('is_trained',index.is_trained)
index.train(xb)
print('is_trained',index.is_trained) 

index.add(xb)                  # 添加索引可能会有一点慢
D, I = index.search(xq, k)     # 搜索
print(I[:5])                   # 最初五次查询的结果
print(I[-5:])                  # 最后五次查询的结果

is_trained False
is_trained True
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900  9309  9810 10048]
 [11055 10895 10812 11321]
 [11353 10164  9787 10719]
 [10571 10664 10632 10203]
 [ 9628  9554  9582 10304]]


结果和上面的强力搜索类似，但是不同。这是因为一些结果不在完全相同的Voronoi。因此，访问更多的单元格可能是有用的。

In [48]:
index.nprobe = 10              # 默认 nprobe 是1 ,可以设置的大一些试试
D, I = index.search(xq, k)
print(I[-5:])                  # 最后五次查询的结果

[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


这是正确的结果。

但注意：在这种情况下获得完美的结果只是数据分布的一个假象，因为它在x轴上有一个强大的组件，这使得它更容易处理。nprobe参数始终是调整速度和结果精度之间权衡的一种方式。设置nprobe = nlist给出与强力搜索相同的结果（但较慢）。

## IndexIVFPQ-更小空间

我们看到的索引IndexFlatL2和IndexIVFFlat都存储完整的向量。 

为了扩展到非常大的数据集，Faiss提供了基于产品量化器的有损压缩来压缩存储的向量的变体。压缩的方法基于乘积量化。

In [49]:
nlist = 100
m = 8
k = 4
quantizer = faiss.IndexFlatL2(d)    # 内部的索引方式依然不变
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8) # 每个向量都被编码为8个字节大小

index.train(xb)
index.add(xb)

index.nprobe = 10                   # 与以前的方法相比
D, I = index.search(xq, k)          
print(I[:5])                        # 最初五次查询的结果
print(I[-5:])                       # 最后五次查询的结果

[[ 399  210  329 1619]
 [1193   39  911  187]
 [1267  197  527  425]
 [ 184  599  466  359]
 [ 828  377  120  416]]
[[ 9900  8746  9853 10437]
 [10494 10507 11373  9014]
 [10719 11291 10424 10138]
 [10122  9638 11113 10630]
 [ 9229 10304  9644 10370]]


In [ ]:
我们可以观察到最近的邻居被正确地找到（它是矢量ID本身），但是向量自身的估计距离不是0，尽管它远远低于与其他邻居的距离。这是由于有损压缩。

另外搜索真实查询时，虽然结果大多是错误的，但是它们在正确的空间区域，而对于真实数据，情况更好，因为：
1. 统一数据很难进行索引，因为没有规律性可以被利用来聚集或降低维度
2. 对于自然数据，语义最近邻居往往比不相关的结果更接近。